# Path explorer

This notebook contains code related to the definition and exploration of the different states that the `MED` can take at different samples in time.

- Directed graph build by defining nodes / vertices that are the states that the FSM of the system can take, and repeated at each time step. Also, the edges (representing transitions between states) connecting these nodes are obtained from the FSM itself, so only valid transitions from the states at each step are considered.
- Path exploration. In order to obtain every possible path for any given initial state and a prediction horizon (max number of steps to evaluate). A Deep First Search (DFS) is performed in a recursive function to find every possibility. The result is a list with every possible path or trajectory for the given initial states. Note that increasing the number of steps can lead to the "tree explosion" due to the exponential growth that takes place in these kind of systems.
- The visualization of these two aspects, is then visualized.

Expected result:

![expected result](docs/attachments/path_viz_med.png)

In [2]:
import pandas as pd
import plotly.graph_objs as go
import numpy as np
from loguru import logger
import json
from pathlib import Path

from solarMED_optimization.path_exploration import Node, Edge, generate_edges, generate_edges_dataframe
from solarMED_modeling import MedState

from phd_visualizations import save_figure

%load_ext autoreload

%autoreload 2

In [3]:
# # g = ig.Graph(n=10, edges=[[0, 1], [2, 3]], directed=True)
# 
# g = ig.Graph.Read_GML('auxiliar/netscience.gml')
# 
# E=[e.tuple for e in g.es]# list of edges
# layt=g.layout('kk') #kamada-kawai layout
# labels=list(g.vs['label'])
# N=len(labels)

In [4]:
# Xn=[layt[k][0] for k in range(N)]
# Yn=[layt[k][1] for k in range(N)]
# Xe=[]
# Ye=[]
# for e in E:
#     Xe+=[layt[e[0]][0],layt[e[1]][0], None]
#     Ye+=[layt[e[0]][1],layt[e[1]][1], None]
# 
# 
# fig=go.Figure()
# 
# trace1=fig.add_trace(
#     go.Scatter(x=Xe,
#                y=Ye,
#                mode='lines',
#                line= dict(color='rgb(210,210,210)', width=1),
#                hoverinfo='none'
#                )
# )
# 
# trace2=fig.add_trace(
#     go.Scatter(x=Xn,
#                y=Yn,
#                mode='markers',
#                name='ntw',
#                marker=dict(symbol='circle-dot',
#                                         size=5,
#                                         color='#6959CD',
#                                         line=dict(color='rgb(50,50,50)', width=0.5)
#                                         ),
#                text=labels,
#                hoverinfo='text'
#                )
# )
# 
# axis_conf=dict(showline=False, # hide axis line, grid, ticklabels and  title
#           zeroline=False,
#           showgrid=False,
#           showticklabels=False,
#           title=''
#           )
# fig.update_layout(
#     title= "Coauthorship network of scientists working on network theory and experiment"+\
# "<br> Data source: <a href='https://networkdata.ics.uci.edu/data.php?id=11'> [1]</a>",
#     font= dict(size=12),
#     showlegend=False,
#     autosize=False,
#     width=800,
#     height=800,
#     xaxis=axis_conf,
#     yaxis=axis_conf,
#     margin=dict(
#         l=40,
#         r=40,
#         b=85,
#         t=100,
#     ),
#     hovermode='closest',
#     annotations=[
#            dict(
#            showarrow=False,
#             text='This igraph.Graph has the Kamada-Kawai layout',
#             xref='paper',
#             yref='paper',
#             x=0,
#             y=-0.1,
#             xanchor='left',
#             yanchor='bottom',
#             font=dict(
#             size=14
#             )
#             )
#         ]
# )
# 
# fig.show()

In [5]:
# from models_psa.fsms import SolarMED, SolarMED_State, MedState
# 
# model = SolarMED(
#     Tts_h = np.zeros((1,3)),
#     Tts_c = np.zeros((1,3)),
#     Tsf_in_ant = np.zeros((1,50)),
#     msf_ant = np.zeros((1,50)),
# )
# 
# [print(f"State {i}: {state.name} {state.value}") for i, state in enumerate(SolarMED_State)]
# 
# [print(state.value) for state in SolarMED_State]

In [6]:
# Build states/nodes/vertices dataframe

Np = 12

base_list = [str(state.value) for state in MedState]
N_nodes = Np*len(base_list)

# Just the SF-TS to test
# Prepend 'step{idx}_' to each string in the base list, repeat N times
result = [
    Node(
        step_idx=step_idx,
        state=state,
    ).model_dump()
    for step_idx in range(Np) for state in [state for state in MedState]
]

nodes_df = pd.DataFrame(result)

display(nodes_df.head())

,step_idx,state,node_id,state_value,state_name,x_pos,y_pos
0,0,MedState.OFF,step000_0,0,OFF,0,0.0
1,0,MedState.GENERATING_VACUUM,step000_1,1,GENERATING_VACUUM,0,1.0
2,0,MedState.IDLE,step000_2,2,IDLE,0,2.0
3,0,MedState.STARTING_UP,step000_3,3,STARTING_UP,0,3.0
4,0,MedState.SHUTTING_DOWN,step000_4,4,SHUTTING_DOWN,0,4.0


In [7]:
# Build transition/connection/edges dataframe

# Procedure:
# Get all possible transitions from the model starting from each state
# How to account for the samples that takes for some of the transitions?
# So, starting from each state, get all possible transitions, and then try to transition to each one of them
# if the transition is successful, then add it to the list of edges, it should be on the form: src_node_id, dst_node_id, line_type solid
# otherwise add the src_node and dst_node where src=dst and line type is dashed.
# Move one step forward and try again until it does. Add the transition to the list of edges (src_node_id, dst_node_id and solid line type)

# from models_psa import MedState, SF_TS_State
# from typing import Literal
# from models_psa.fsms import SolarFieldWithThermalStorage_FSM, MedFSM
# 
# 
# def generate_edges(result_list: list[dict], step_idx: int, subsystem:Literal['sf-ts', 'med'], Np: int):
# 
#     """
#     Generate edges for the FSM of the subsystem
# 
#     Aspects to consider:
#         - By naming convention, long transitions are triggered with a transition/trigger id that starts with 'start_', and finishes with 'finish_'. When a transition is in progress, the trigger id is 'inprogress_' and is blocked from being triggered again until the transition is completed.
# 
#     :param result_list: 
#     :param step_idx: 
#     :param subsystem: 
#     :return: 
#     """
# 
#     if subsystem.lower() == 'sf-ts':
#         # Let's start with SF-TS which is simpler
#         for state in SF_TS_State:
#             result = {}
#             model = SolarFieldWithThermalStorage_FSM(initial_state=state, sample_time=1)
# 
#             result = {
#                 'step_idx': step_idx,
#                 'src_node_id': f'step{step_idx:03d}_{state.value}',
#                 'src_node_name': state.name,
#                 'line_type': 'solid'
#             }
# 
#             triggers = model.machine.get_triggers(state)
#             for trigger in triggers:
#                 # Always add the option to remain in the same state
#                 # how to get the destination state?
#                 dst_state = model.machine.get_transitions(trigger)[0].dest
#                 dst_state = model.machine.get_state(dst_state)
# 
#                 result.update({
#                     'dst_node_id': f'step{step_idx+1:03d}_{dst_state.value}',
#                     'dst_node_name': dst_state.name,
#                     'transition_id': trigger
#                 })
# 
#             # Always add the option to remain in the same state
#             result.update({
#                 'dst_node_id': f'step{step_idx+1:03d}_{state.value}',
#                 'dst_node_name': state.name,
#                 'transition_id': 'none'
#             })
# 
#             result_list.append(result)
# 
#         return result_list
# 
#     if subsystem.lower() == 'med':
#         for state in MedState:
#             model = MedFSM(
#                 initial_state=state, sample_time=1, 
#                 vacuum_duration_time=5, brine_emptying_time=2, startup_duration_time=2
#             )
# 
#             triggers = model.machine.get_triggers(state)
#             for trigger in triggers:
# 
#                 # Check if a long transition was already included in a previous iteration
#                 # Check for the src_state and step_idx
#                 # if any([
#                 #     i['src_node_name'] == state.name and 
#                 #     i['step_idx'] == step_idx and 
#                 #     'inprogress' in i['transition_id'] 
#                 #     for i in result_list
#                 # ]):
#                 #     logger.debug(f"Transition {trigger} from {state.name} blocked by call in previous iteration, skipping")
#                 #     continue
#                 # Removed, it was a mistake since it can always be that the transition was not triggered previously and so it's triggered in this specific step
# 
# 
#                 # print(f'{state.name} -> {trigger}')
#                 dst_state = model.machine.get_transitions(trigger)[0].dest
#                 dst_state = getattr(MedState, dst_state)
# 
#                 duration = 1
# 
#                 if state == MedState.STARTING_UP:
#                     duration = model.startup_duration_samples
# 
#                 elif state == MedState.GENERATING_VACUUM:
#                     duration = model.vacuum_duration_samples
# 
#                 elif state == MedState.SHUTTING_DOWN:
#                     duration = model.brine_emptying_samples
# 
#                 # Cancel transitions are instantaneous
#                 if 'cancel' in trigger: duration = 1
# 
#                 for idx in range(step_idx, step_idx+duration-1):
#                     if idx+1 < Np:
# 
#                         result_list.append({
#                             'step_idx': idx,
#                             'src_name': state.name,
#                             'dst_name': state.name,
#                             'src_node_id': int(f'{idx}{state.value}'),
#                             'dst_node_id': int(f'{idx+1}{state.value}'),
#                             'src_node_name': f'step{idx:03d}_{state.value}', 
#                             'dst_node_name': f'step{idx+1:03d}_{state.value}',
#                             'transition_id': f'step{idx:03d}_'+trigger.replace('finish', 'inprogress')+f'_since_step{step_idx}',  # Name convention
#                             'line_type': 'dash'
#                         })
# 
#                 if step_idx+duration < Np:
# 
#                     # In the last step, the transition is completed  
#                     result_list.append({
#                         'step_idx': step_idx+duration-1,
#                         'src_name': state.name,
#                         'dst_name': dst_state.name,
#                         'src_node_id': int(f'{step_idx+duration-1}{state.value}'),
#                         'dst_node_id': int(f'{step_idx+duration}{dst_state.value}'),
#                         'src_node_name': f'step{step_idx+duration-1:03d}_{state.value}', 
#                         'dst_node_name': f'step{step_idx+duration:03d}_{dst_state.value}',
#                         'transition_id': f'step{step_idx+duration-1:03d}_'+trigger if duration==1 else f'step{step_idx+duration-1:03d}_{trigger}_from_step{step_idx}', 
#                         'line_type': 'solid'
#                     })
# 
#             if step_idx+1 < Np:
#                 # Always add the option to remain in the same state
#                 result_list.append({
#                     'step_idx': step_idx,
#                     'src_name': state.name,
#                     'dst_name': state.name,
#                     'src_node_id': int(f'{step_idx}{state.value}'),
#                     'dst_node_id': int(f'{step_idx+1}{state.value}'),
#                     'src_node_name': f'step{step_idx:03d}_{state.value}', 
#                     'dst_node_name': f'step{step_idx+1:03d}_{state.value}',
#                     'transition_id': f'step{step_idx:03d}_none',
#                     'line_type': 'solid'
#                 })
# 
#         return result_list
# 
# 
#     raise NotImplementedError(f'Unsupported subsystem {subsystem}')
# 
# 
# def generate_dataframe(edges_dict: dict) -> pd.DataFrame:
# 
#     df = pd.DataFrame(edges_dict)
# 
#     # Make sure the first column is the src_node_id and the second is the dst_node_id
#     cols = ['src_node_id', 'dst_node_id'] + [col for col in df.columns if col not in ['src_node_id', 'dst_node_id']]
#     df = df.reindex(columns=cols)
# 
#     return df
# 
# a = generate_edges([], 0, 'med', Np=10)
# a = generate_edges(a, 1, 'med', Np=10)
# 
# for i in a:
#     print(f"[{i['step_idx']}] {i['src_name']} --{i['transition_id']}--> {i['dst_name']}")
    

In [8]:
# step_idx = 0
# Np = 24
# 
# result = []; result2 = []
# for step_idx in range(Np):
#     result = generate_edges(result, step_idx, 'sf-ts', Np=Np)
#     result2 = generate_edges(result2, step_idx, 'med', Np=Np)
# 
# edges_sf_ts = generate_dataframe(result)
# edges_med = generate_dataframe(result2)
# 
# display(edges_sf_ts.head())
# display(edges_med.head())

In [9]:
step_idx = 0

edges_list = []
for step_idx in range(Np):
    edges_list = generate_edges(edges_list, step_idx, system='MED', Np=Np)
    
# Convert to dataframe
edges_df = generate_edges_dataframe(edges_list)

edges_df.head()

,src_node_id,dst_node_id,step_idx,src_name,dst_name,transition_id,line_type,x_pos_src,x_pos_dst,y_pos_src,y_pos_dst
0,step000_0,step001_1,0,OFF,GENERATING_VACUUM,step000_start_generating_vacuum,solid,None,None,None,None
1,step000_0,step001_0,0,OFF,OFF,step000_none,solid,None,None,None,None
2,step000_1,step001_1,0,GENERATING_VACUUM,GENERATING_VACUUM,step000_inprogress_generating_vacuum_since_step0,dash,None,None,None,None
3,step001_1,step002_1,1,GENERATING_VACUUM,GENERATING_VACUUM,step001_inprogress_generating_vacuum_since_step0,dash,None,None,None,None
4,step002_1,step003_1,2,GENERATING_VACUUM,GENERATING_VACUUM,step002_inprogress_generating_vacuum_since_step0,dash,None,None,None,None


In [9]:
# Checks

# Get unique source and target node IDs from the edges DataFrame
edge_node_ids = pd.concat([edges_df['src_node_id'], edges_df['dst_node_id']]).unique()

# Get node IDs from the vertices DataFrame
vertex_node_ids = nodes_df['node_id'].unique()

# Find the difference between the two sets
missing_node_ids = set(edge_node_ids) - set(vertex_node_ids)
missing_node_ids2 = set(vertex_node_ids) - set(edge_node_ids)

print(f"Missing node IDs: {missing_node_ids}, {missing_node_ids2}")

Missing node IDs: set(), set()


## Load previously DFS generated paths
By running `recursitron.py` the paths were generated and saved in a JSON file. Now we load them and visualize them.

Done this way to avoid the performance penalty of evaluating the function inside a Jupyter notebook.

In [10]:
from solarMED_optimization import convert_to_state

# converters = {col_name: convert_to_state for col_name in pd.read_csv('results/all_paths.csv', nrows=0).columns}
# 
# all_paths = pd.read_csv('results/all_paths.csv', converters=converters)

# Read json file
with open('results/all_paths_MED.json', 'r') as f:
    all_paths = json.load(f)
    
# Convert to the correct type
all_paths = [[convert_to_state(state) for state in path] for path in all_paths] # Terrible

### Visualize the directed graph

In [11]:
# Create the coordinates for the edges

# edges_df[
#     ['x_pos_src', 'x_pos_dst', 'y_pos_src', 'y_pos_dst']
# ] = pd.DataFrame(np.zeros((len(edges_df), 4)), index=edges_df.index)
# 
# for idx, row in edges_df.iterrows():
#     # Find every row in the edges DataFrame that has the same src_node_name as the current node
#     
#     src_node_name = row['src_node_name']
#     dst_node_name = row['dst_node_name']
#     
#     src_node = nodes_df[nodes_df['name'] == src_node_name]
#     dst_node = nodes_df[nodes_df['name'] == dst_node_name]
#     
#     if len(src_node) >1 or len(dst_node) > 1:
#         raise RuntimeError(f"Multiple nodes with the same name {src_node_name} / {dst_node_name} found")
#     
#     if len(src_node) == 0 or len(dst_node) == 0:
#         raise RuntimeError(f"No nodes with the name {src_node_name} / {dst_node_name} found")
# 
#     src_node = src_node.iloc[0]; dst_node = dst_node.iloc[0]
# 
#     increment = 0.01 * src_node['state_value'] if row['line_type'] == 'dash' else 0
# 
#     if dst_node['x_pos'] - src_node['x_pos'] > 1:
#         raise RuntimeError(f"More than one transition duration from {src_node_name} to {dst_node_name}")
# 
#     edges_df.loc[idx, 'x_pos_src'] = src_node['x_pos']
#     edges_df.loc[idx, 'x_pos_dst'] = dst_node['x_pos'] - 0.1 # So the arrow is not on top of the node circle
#     edges_df.loc[idx, 'y_pos_src'] = src_node['y_pos'] + increment
#     edges_df.loc[idx, 'y_pos_dst'] = dst_node['y_pos'] + increment


In [12]:
len(all_paths[0])

12

In [12]:
# # Build the graph

# 
# options_avg = round( len(all_paths) / len(MedState) ) 
# # def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
# #     L=len(pos)
# #     if len(text)!=L:
# #         raise ValueError('The lists pos and text must have the same len')
# #     annotations = []
# #     for k in range(L):
# #         annotations.append(
# #             dict(
# #                 text=labels[k], # or replace labels with a different list for the text within the circle
# #                 x=pos[k][0], y=2*M-position[k][1],
# #                 xref='x1', yref='y1',
# #                 font=dict(color=font_color, size=font_size),
# #                 showarrow=False)
# #         )
# #     return annotations
# 
# # Build vectors in the format wanted by plotly ([xsrc_0, xdst_0, None, xsrc_1, xdst_1, None, ...] and [ysrc_0, ydst_0, None, ysrc_1, ydst_1, None, ...])
# Xe = []; Xe_solid = []; Xe_dash = []
# Ye = []; Ye_solid = []; Ye_dash = []
# for idx, row in edges_df.iterrows():
#     Xe += [row['x_pos_src'], row['x_pos_dst'], None]
#     Ye += [row['y_pos_src'], row['y_pos_dst'], None]
#     
#     Xe_solid += [row['x_pos_src'], row['x_pos_dst'], None] if row['line_type'] == 'solid' else []
#     Ye_solid += [row['y_pos_src'], row['y_pos_dst'], None] if row['line_type'] == 'solid' else []
#     
#     Xe_dash += [row['x_pos_src'], row['x_pos_dst'], None] if row['line_type'] == 'dash' else []
#     Ye_dash += [row['y_pos_src'], row['y_pos_dst'], None] if row['line_type'] == 'dash' else []
# 
# fig=go.FigureWidget()
# 
# # fig.add_trace(
# #     go.Scatter(
# #         x=Xe,
# #         y=Ye,
# #         mode='lines',
# #         line= dict(color='rgb(210,210,210)', width=1, dash=edges_df['line_type'].values.tolist()),
# #         hoverinfo='none'
# #     )
# # )
# 
# 
# fig.add_trace(
#     go.Scatter(
#         x=nodes_df['x_pos'].values,
#         y=nodes_df['y_pos'].values,
#         mode='markers',
#         name='states',
#         marker=dict(symbol='circle-dot', size=20, color='#6959CD'),
#         line=dict(color='rgb(50,50,50)', width=0.5),
#         text=nodes_df['state_name'].values,
#         hoverinfo='text'
#     )
# )
# 
# # nodes_scatter = fig.data[-1]
# 
# # Create separate traces for solid and dashed lines
# fig.add_trace(
#     go.Scatter(
#         x=Xe_solid,
#         y=Ye_solid,
#         mode='lines+markers',
#         line=dict(color='rgb(210,210,210)', width=1, dash='solid'),
#         marker=dict(symbol='arrow', size=10, color='rgb(210,210,210)', angleref="previous"),
#         text=edges_df['transition_id'].values,
#         hoverinfo='text',
#     )
# )
# 
# fig.add_trace(
#     go.Scatter(
#         x=Xe_dash,
#         y=Ye_dash,
#         mode='lines+markers',
#         line=dict(color='rgb(210,210,210)', width=1, dash='dash'),
#         marker=dict(symbol='arrow', size=10, color='rgb(210,210,210)', angleref="previous"),
#         hoverinfo='none'
#     )
# )
# 
# # Empty scatter to be used for highlighting arriving edges
# fig.add_trace(
#     go.Scatter(
#         x=[], y=[], hoverinfo='none',
#         mode='lines+markers', line=dict(color='#06C892', width=3, dash='solid'),
#         marker=dict(symbol='arrow', size=10, color='#06C892', angleref="previous"),
#     )
# )
# # Empty scatter to be used for highlighting departing edges
# fig.add_trace(
#     go.Scatter(
#         x=[], y=[], hoverinfo='none',
#         mode='lines+markers', line=dict(color='#AD72F3', width=3, dash='solid'),
#         marker=dict(symbol='arrow', size=10, color='#AD72F3', angleref="previous"),
#     )
# )
# 
# 
# axis_conf=dict(
#     showline=False, # hide axis line, grid, ticklabels and  title
#     zeroline=False,
#     showgrid=False,
#     title=''
# )
# 
# fig.update_yaxes(
#     ticktext=[state.name for state in MedState],
#     tickvals=[state.value for state in MedState],
# )
# 
# fig.update_xaxes(
#     tickvals=np.arange(Np, step=1),
# )
# 
# fig.update_layout(
#     title= "Directed graph of the operating modes evolution in the MED plant"+\
#         f"<br> Average number of alternative paths per state {options_avg}</br>",
# # "<br> Data source: <a href='https://networkdata.ics.uci.edu/data.php?id=11'> [1]</a>",
#     font=dict(size=12),
#     showlegend=False,
#     autosize=False,
#     width=1400,
#     height=500,
#     xaxis=dict(
#         zeroline=False,
#         showline=False,
#         showgrid=False,
#         showticklabels=True,
#         title='Time steps'
#     ),
#     yaxis=axis_conf,
#     margin=dict(
#         l=40,
#         r=40,
#         b=85,
#         t=100,
#     ),
#     hovermode='closest',
#     # annotations=[
#     #    dict(
#     #        showarrow=False,
#     #         text='This igraph.Graph has the Kamada-Kawai layout',
#     #         xref='paper', yref='paper',
#     #         x=0, y=-0.1,
#     #         xanchor='left', yanchor='bottom',
#     #         font=dict(size=14)
#     #     )
#     # ]
# )
# 
# # nodes_scatter.on_click(highlight_node_paths)
# 
# def get_coordinates(node_id: str, type: Literal['src', 'dst']):
#     node_type = "dst" if type=="dst" else "src"
#     edges = edges_df[edges_df[f'{node_type}_node_name'] == node_id]
#     x_src_aux = edges['x_pos_src'].values
#     x_dst_aux = edges['x_pos_dst'].values
#     y_src_aux = edges['y_pos_src'].values
#     y_dst_aux = edges['y_pos_dst'].values
#     
#     x = []; y = []
#     for xsrc, xdst, ysrc, ydst in zip(x_src_aux, x_dst_aux, y_src_aux, y_dst_aux):
#         x += [xsrc, xdst, None]
#         y += [ysrc, ydst, None]
#         
#     return x, y
# 
# # @interact(step_idx=(0, Np-1, 1), state=[state.name for state in MedState])
# # def update(step_idx=6, state=MedState.IDLE.name):
# #     
# #     with fig.batch_update():
# #         # scatt.x=xs
# #         # scatt.y=np.sin(a*xs-b)
# #         # scatt.line.color=color
# #         # fig.data[0].x=xs
# #         # fig.data[0].y=np.sin(a*xs-b)
# #         # Find the node with the name and step_idx
# #         node_id = f'step{step_idx:03d}_{getattr(MedState, state).value}'
# #         
# #         x, y = get_coordinates(node_id, 'src')
# #         fig.data[-1].x = x
# #         fig.data[-1].y = y
# #         
# #         x, y = get_coordinates(node_id, 'dst')
# #         fig.data[-2].x = x
# #         fig.data[-2].y = y
# #         
# #         # Would be cool to add an annotation to the edges with the transition id
# #         
# #     # print(f"a={a}, b={b}, color={color}")
# #     
# def get_coordinates_edge(src_node_id: str, dst_node_id: str) -> tuple[list[float], list[float]]:
#     src_node = nodes_df[nodes_df['name'] == src_node_id]
#     dst_node = nodes_df[nodes_df['name'] == dst_node_id]
# 
#     if len(src_node) >1 or len(dst_node) > 1:
#         raise RuntimeError(f"Multiple nodes with the same name {src_node_id} / {dst_node_id} found")
# 
#     return (
#         [src_node['x_pos'].values[0], dst_node['x_pos'].values[0], None], 
#         [src_node['y_pos'].values[0], dst_node['y_pos'].values[0], None]
#     )
# 
# 
# @interact(initial_state=[state for state in MedState], option_idx=(0, options_avg, 1))
# def add_path_highlight(initial_state=MedState.ACTIVE, option_idx=0):
#     
#     if initial_state is None:
#         # Use random module to choose an integer from 0 to len(all_paths)
#         path_idx = np.random.randint(0, len(all_paths))
#     
#     else :
#         # Find the path that starts from the initial state
#         path_idx = [idx for idx, path in enumerate(all_paths) if path[0] == initial_state]
#         path_idx = path_idx[option_idx] if option_idx < len(path_idx) else path_idx[-1]
#         print(f"Selected path {path_idx}: {[state.name for state in all_paths[path_idx]]}")
#     
#     # Somehow build the path coordinates from the list of all_paths
#     path = all_paths[path_idx]
#     
#     x = []; y = []
#     for step_idx in range(0, len(path)-1, 1):        
#         # If the naming scheme changes for whatever reason it will break
#         src_node_id = f'step{step_idx:03d}_{path[step_idx].value}'
#         dst_node_id = f'step{step_idx+1:03d}_{path[step_idx+1].value}'
#         
#         x_aux, y_aux = get_coordinates_edge(src_node_id, dst_node_id)
#         
#         x += x_aux
#         y += y_aux
#     
#     with fig.batch_update():
#         # First deactivate departing highlights 
#         fig.data[-2].x = None
#         fig.data[-2].y = None
#         
#         # Then use arriving trace container to include the path
#         fig.data[-1].x = x
#         fig.data[-1].y = y
#         
#     
#     
#     

from ipywidgets import interact, widgets
from solarMED_optimization.path_exploration import generate_edges_coordinates
from solarMED_optimization.visualization import plot_state_graph, get_coordinates_edge

generate_edges_coordinates(nodes_df, edges_df)

fig = plot_state_graph(nodes_df, edges_df=edges_df, system='MED', Np=len(all_paths[0]), width=900)

options_avg = round(len(all_paths) / len(MedState))
base_title = "Directed graph of the operating modes evolution in the MED system"


@interact(initial_state=[state for state in MedState], option_idx=(0, options_avg, 1))
def add_path_highlight(initial_state=MedState.IDLE, option_idx=0):

    if initial_state is None:
        # Use random module to choose an integer from 0 to len(all_paths)
        path_idx = np.random.randint(0, len(all_paths))

    else:
        # Find the path that starts from the initial state
        path_idx = [idx for idx, path in enumerate(all_paths) if path[0] == initial_state]
        path_idx = path_idx[option_idx] if option_idx < len(path_idx) else path_idx[-1]
        print(f"Selected path {path_idx}: {[state.name for state in all_paths[path_idx]]}")

    # Somehow build the path coordinates from the list of all_paths
    path = all_paths[path_idx]

    x = []
    y = []
    for step_idx in range(0, len(path) - 1, 1):
        # If the naming scheme changes for whatever reason it will break
        src_node_id = f'step{step_idx:03d}_{path[step_idx].value}'
        dst_node_id = f'step{step_idx + 1:03d}_{path[step_idx + 1].value}'

        x_aux, y_aux = get_coordinates_edge(src_node_id, dst_node_id, nodes_df=nodes_df)

        x += x_aux
        y += y_aux

    with fig.batch_update():
        # First deactivate departing highlights
        fig.data[-2].x = None
        fig.data[-2].y = None

        # Then use arriving trace container to include the path
        fig.data[-1].x = x
        fig.data[-1].y = y
        
        fig.layout.title.text = f'<b>{base_title}</b><br><span style="font-size: 11px;">Selected path {path_idx}: {[state.name for state in all_paths[path_idx]]}</span></br>'

# Add a button that will highlight the paths that arrive/leave to/from a random node
# button = widgets.Button(description='Highlight random path')
# 
# button.on_click(add_path_highlight)
# 
fig
# widgets.VBox([fig, button])
# 
# add_path_highlight(None)

interactive(children=(Dropdown(description='initial_state', index=2, options=(<MedState.OFF: 0>, <MedState.GEN…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(210,210,210)', 'dash': 'solid', 'width': 1},
              'marker': {'angleref': 'previous', 'color': 'rgb(210,210,210)', 'size': 10, 'symbol': 'arrow'},
              'mode': 'lines+markers',
              'text': array(['step000_start_generating_vacuum', 'step000_none',
                             'step000_inprogress_generating_vacuum_since_step0', ..., 'step010_none',
                             'step010_start_shutdown', 'step010_none'], dtype=object),
              'type': 'scatter',
              'uid': '373d95c5-ae94-4389-b04c-24ea3904a4fe',
              'x': [0, 0.9, None, ..., 10, 10.9, None],
              'y': [0.0, 1.0, None, ..., 5.0, 5.0, None]},
             {'hoverinfo': 'none',
              'line': {'color': 'rgb(210,210,210)', 'dash': 'dash', 'width': 1},
              'marker': {'angleref': 'previous', 'color': 'rgb(210,210,210)', 'size': 10, 'symbol': 'arrow'},
              '

In [18]:
# Save figure
save_figure(fig, 'MED_state_evaluation_example', 'docs/attachments',
            formats= ['html', 'svg'], width=fig.layout.width, height=fig.layout.height)

2024-05-06 13:42:30.697 | INFO     | phd_visualizations:save_figure:33 - Figure saved in ['docs/attachments']/MED_state_evaluation_example.html
2024-05-06 13:42:30.952 | INFO     | phd_visualizations:save_figure:33 - Figure saved in ['docs/attachments']/MED_state_evaluation_example.svg


### Generate a list for every possible path
for every possible initial state

In [14]:
# from models_psa.fsms import MedFSM
# import inspect
# from transitions import Machine
# 
# model = MedFSM(sample_time=1, vacuum_duration_time=10, brine_emptying_time=2, startup_duration_time=2)
# 
# # Define expected inputs to the step method
# expected_inputs_keys = list(inspect.signature(model.step).parameters.keys())
# expected_inputs_default = {arg: None for arg in expected_inputs_keys}
# 
# print(model.machine.get_triggers('IDLE')) # -> start_startup
# print(model.machine.get_transitions(trigger='start_startup'))
# 
# transition = model.machine.get_transitions(trigger='start_shutdown')[0]
# 
# # For a particular transition, get its expected inputs to be satisfied
# condition_obj = transition.conditions[0]
# condition_method = getattr(model, condition_obj.func)
# 
# if condition_obj.target == False:
#     expected_inputs = condition_method(return_invalid_inputs=True)
# else:
#     expected_inputs = condition_method(return_valid_inputs=True)
# 
# # Update expected_inputs with default values when missing
# for key, value_default in expected_inputs_default.items():
#     if key not in expected_inputs:
#         expected_inputs[key] = value_default
# 
# print(expected_inputs)

In [15]:
from recursitron import get_all_paths, dump_as

all_paths = get_all_paths(
    system='MED',
    machine_init_args={
        'sample_time': 1,
        'vacuum_duration_time': 3,
        'brine_emptying_time': 1,
        'startup_duration_time': 1
    },
    max_step_idx=Np,
    # initial_states=[MedState.IDLE, MedState.OFF],
    use_parallel=False
)

dump_as(all_paths, Path(f'results/all_paths_MED_Np_{Np}'), file_format='json')

2024-04-29 13:36:26.625 | INFO     | recursitron:generate_all_paths:124 - Hello, this is recursitron 000, current state is: MedState.OFF, current step: 0 and we are 2 machines deep with samples [0, 0]
2024-04-29 13:36:26.625 | INFO     | recursitron:generate_all_paths:148 - Transition: start_generating_vacuum
2024-04-29 13:36:26.626 | DEBUG    | models_psa.fsms:inform_exit_state:140 - Left state OFF
2024-04-29 13:36:26.626 | INFO     | models_psa.fsms:set_vacuum_start:486 - Started generating vacuum, it will take 3 samples to complete
2024-04-29 13:36:26.627 | DEBUG    | models_psa.fsms:inform_enter_state:136 - Entered state GENERATING_VACUUM
2024-04-29 13:36:26.628 | INFO     | recursitron:generate_all_paths:124 - Hello, this is recursitron 001, current state is: MedState.GENERATING_VACUUM, current step: 1 and we are 3 machines deep with samples [0, 1, 1]
2024-04-29 13:36:26.628 | INFO     | recursitron:generate_all_paths:148 - Transition: finish_generating_vacuum
2024-04-29 13:36:26.

# Others

In [24]:
nodes_scatter.marker.color

c = list(nodes_scatter.marker.color)
s = list(nodes_scatter.marker.size)

TypeError: 'int' object is not iterable

In [2]:
import plotly.graph_objs as go
import numpy as np
from ipywidgets import interact


fig = go.FigureWidget()
scatt = go.Scatter(
    x=np.linspace(0, 6, 100),
    y=np.sin(3.6*np.linspace(0, 6, 100)-4.3),
    mode='lines',
    line=dict(color='blue')
)
        
fig.add_trace(scatt)

xs=np.linspace(0, 6, 100)

@interact(a=(1.0, 4.0, 0.01), b=(0, 10.0, 0.01), color=['red', 'green', 'blue'])
def update(a=3.6, b=4.3, color='blue'):
    with fig.batch_update():
        # scatt.x=xs
        # scatt.y=np.sin(a*xs-b)
        # scatt.line.color=color
        fig.data[0].x=xs
        fig.data[0].y=np.sin(a*xs-b)
        fig.data[0].line.color=color
        
    print(f"a={a}, b={b}, color={color}")

fig

interactive(children=(FloatSlider(value=3.6, description='a', max=4.0, min=1.0, step=0.01), FloatSlider(value=…

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'type': 'scatter',
              'uid': '3c6ce09a-bc84-4552-bed4-d46081e8c42a',
              'x': array([0.        , 0.06060606, 0.12121212, 0.18181818, 0.24242424, 0.3030303 ,
                          0.36363636, 0.42424242, 0.48484848, 0.54545455, 0.60606061, 0.66666667,
                          0.72727273, 0.78787879, 0.84848485, 0.90909091, 0.96969697, 1.03030303,
                          1.09090909, 1.15151515, 1.21212121, 1.27272727, 1.33333333, 1.39393939,
                          1.45454545, 1.51515152, 1.57575758, 1.63636364, 1.6969697 , 1.75757576,
                          1.81818182, 1.87878788, 1.93939394, 2.        , 2.06060606, 2.12121212,
                          2.18181818, 2.24242424, 2.3030303 , 2.36363636, 2.42424242, 2.48484848,
                          2.54545455, 2.60606061, 2.66666667, 2.72727273, 2.78787879, 2.84848485,
                          2.9090909

In [22]:
import plotly.graph_objects as go

import numpy as np
np.random.seed(1)

x = np.random.rand(100)
y = np.random.rand(100)

f = go.FigureWidget([go.Scatter(x=x, y=y, mode='markers')])

scatter = f.data[0]
colors = ['#a3a7e4'] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
f.layout.hovermode = 'closest'


# create our callback function
def update_point(trace, points, selector):
    
    logger.info('hola')
    
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    for i in points.point_inds:
        c[i] = '#bae2be'
        s[i] = 20
        with f.batch_update():
            scatter.marker.color = c
            scatter.marker.size = s


scatter.on_click(update_point)

f

FigureWidget({
    'data': [{'marker': {'color': [#a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                         